# HDBSCAN Geospatial Clustering

## Loading Libraries & Data

In [1]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import hdbscan
from hdbscan import approximate_predict

In [2]:
# Loading CSV
listings = pd.read_csv(r"..\data\listings_cleaner.csv")

## Preprocessing for Clustering

In [3]:
# Filtering listings above $1800 (Massive price outliers)
listings = listings[listings["price"] <= 1800]

In [4]:
# Pasting drops and renames
listings = listings.drop(columns = 
                ["Unnamed: 0.1", "Unnamed: 0", "host_id", "id", # Removing index columns and IDs
                "name", "last_scrape_period", "description", "last_scraped"]) # Removing free text columns

# Renaming long column names to make them easier to work with
listings = listings.rename(columns = {
    "accommodates": "accomm", "instant_bookable": "instant_book",
    "neighbourhood_group_cleansed": "boro", "host_neighbourhood": "nbhd",
    "host_has_profile_pic": "has_pfp", "host_tenure": "tenure", "host_is_superhost": "is_superhost",
    "host_identity_verified": "id_ver", "host_listings_count": "lst_cnt", "host_total_listings_count": "total_lst_cnt",
    "host_response_rate": "rsp_rate", "host_response_time": "rsp_time", "host_acceptance_rate": "accept_rate",
    "availability_30": "avail_30", "availability_365": "avail_365", "has_availability": "has_avail",
    "maximum_nights": "max_nights", "minimum_nights": "min_nights",
    "number_of_reviews": "no_rev", "review_scores_accuracy": "rev_acc", "review_scores_checkin": "rev_checkin",
    "review_scores_cleanliness": "rev_clean", "review_scores_communication": "rev_coms", "review_scores_location": "rev_loc",
    "review_scores_value": "rev_val", "review_scores_rating": "rev_rating",
    "scrape_2024-10": "oct", "scrape_2024-11": "nov", "scrape_2024-12": "dec", "scrape_2025-01": "jan",
    "scrape_2025-02": "feb", "scrape_2025-03": "mar", "scrape_2025-04": "apr", "scrape_2025-05": "may", 
    "scrape_2025-06": "june", "scrape_2025-07": "jul", "scrape_2025-08": "aug", 
    "prop_apartment": "prop_apt", "prop_entire home/apt": "prop_entire_home_apt","prop_hotel room": "prop_hotel_room",
    "prop_private room": "prop_private_room", "prop_shared room": "prop_shared_room",
    "latitude": "lat", "longitude": "lon"
})

In [5]:
# Viewing exact borough names
listings["boro"].unique()

array(['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island'],
      dtype=object)

## Queens Clustering

In [6]:
# Subsetting to only Queens
qns_df = listings[listings["boro"] == "Queens"].copy()

# Train-Test Splits by index
qns_train_idx, qns_test_idx = train_test_split(qns_df.index, train_size = 0.8, random_state = 2025)
qns_train = qns_df.loc[qns_train_idx].copy()
qns_test  = qns_df.loc[qns_test_idx].copy()

# Getting Lat/Lon as NumPy Arrays
X_train = qns_train[['lat', 'lon']].values
X_test  = qns_test[['lat', 'lon']].values

# Scaling Lat/Lon coordinates
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [7]:
# Getting unique neighborhoods
print(qns_df["nbhd"].nunique()) # 131 unique neighborhoods in Queens
print(len(qns_df)) # 38173 unique listings in Queens
print((len(qns_df) / len(listings)) * 100) # 15.67% of listings are in Queens

qns_cluster_size = 16 # Setting to 16 clusters for about the percentage of listings

131
38173
15.672870451344837


In [8]:
# Fitting HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size = qns_cluster_size, 
                        metric = "euclidean",  # Euclidean metric best for Queens - too small of a geo for haversine. Not grid-like for manhattan.
                        prediction_data = True, 
                        cluster_selection_epsilon = 0.05)

cluster_labels_train = cluster.fit_predict(X_train_scaled)

c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [9]:
# Assigning cluster labels to training data
qns_train["cluster"] = cluster_labels_train

# Assigning cluster labels to test data with approximate predictions
cluster_labels_test, _ = hdbscan.approximate_predict(cluster, X_test_scaled)
qns_test['cluster'] = cluster_labels_test

## Brooklyn Clustering

In [10]:
# Subsetting to only Brooklyn
bk_df = listings[listings["boro"] == "Brooklyn"].copy()

# Train-Test Splits by index
bk_train_idx, bk_test_idx = train_test_split(bk_df.index, train_size = 0.8, random_state = 2025)
bk_train = bk_df.loc[bk_train_idx].copy()
bk_test  = bk_df.loc[bk_test_idx].copy()

# Getting Lat/Lon as NumPy Arrays
X_train = bk_train[['lat', 'lon']].values
X_test  = bk_test[['lat', 'lon']].values

# Scaling Lat/Lon coordinates
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [11]:
# Getting unique neighborhoods
print(bk_df["nbhd"].nunique()) # 212 unique neighborhoods in Brooklyn
print(len(bk_df)) # 84393 unique listings in Brooklyn
print((len(bk_df) / len(listings)) * 100) # 34.65% of listings are in Brooklyn

bk_cluster_size = 35 # Setting the number of clusters to the percentage of listings

212
84393
34.649636025472056


In [12]:
# Fitting HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size = bk_cluster_size, 
                        metric = "euclidean",  # Euclidean metric best for Brooklyn - too small of a geo for haversine. Not grid-like for manhattan.
                        prediction_data = True, 
                        cluster_selection_epsilon = 0.05) 

cluster_labels_train = cluster.fit_predict(X_train_scaled)

c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [13]:
# Assigning cluster labels to training data
bk_train["cluster"] = cluster_labels_train

# Assigning cluster labels to test data with approximate predictions
cluster_labels_test, _ = hdbscan.approximate_predict(cluster, X_test_scaled)
bk_test["cluster"] = cluster_labels_test

## The Bronx Clustering

In [14]:
# Subsetting to only The Bronx
bx_df = listings[listings["boro"] == "Bronx"].copy()

# Train-Test Splits by index
bx_train_idx, bx_test_idx = train_test_split(bx_df.index, train_size = 0.8, random_state = 2025)
bx_train = bx_df.loc[bx_train_idx].copy()
bx_test  = bx_df.loc[bx_test_idx].copy()

# Getting Lat/Lon as NumPy Arrays
X_train = bx_train[["lat", "lon"]].values
X_test  = bx_test[["lat", "lon"]].values

# Scaling Lat/Lon coordinates
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [15]:
# Getting unique neighborhoods
print(bx_df["nbhd"].nunique()) # 86 unique neighborhoods in The Bronx
print(len(bx_df)) # 10003 unique listings in The Bronx
print((len(bx_df) / len(listings)) * 100) # 4.11% of listings are in The Bronx

bx_cluster_size = 8 # Setting min clusters to twice the percentage of listings in the borough

86
10003
4.106979360406633


In [16]:
# Fitting HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size = bx_cluster_size, 
                        metric = "euclidean", # Euclidean metric best for The Bronx - too small of a geo for haversine. Not grid-like for manhattan.
                        prediction_data = True, 
                        cluster_selection_epsilon = 0.05) 

cluster_labels_train = cluster.fit_predict(X_train_scaled)

c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [17]:
# Assigning cluster labels to training data
bx_train["cluster"] = cluster_labels_train

# Assigning cluster labels to test data with approximate predictions
cluster_labels_test, _ = hdbscan.approximate_predict(cluster, X_test_scaled)
bx_test["cluster"] = cluster_labels_test

## Staten Island Clustering

In [18]:
# Subsetting to only Staten Island
st_df = listings[listings["boro"] == "Staten Island"].copy()

# Train-Test Splits by index
st_train_idx, st_test_idx = train_test_split(st_df.index, train_size = 0.8, random_state = 2025)
st_train = st_df.loc[st_train_idx].copy()
st_test  = st_df.loc[st_test_idx].copy()

# Getting Lat/Lon as NumPy Arrays
X_train = st_train[['lat', 'lon']].values
X_test  = st_test[['lat', 'lon']].values

# Scaling Lat/Lon coordinates
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [19]:
# Getting unique neighborhoods
print(st_df["nbhd"].nunique()) # 52 unique neighborhoods in Staten Island
print(len(st_df)) # 3495 unique listings in Staten Island
print((len(st_df) / len(listings)) * 100) # Only 1.44% of listings are in Staten Island

st_cluster_size = 4 # Very few cluster sizes based on the custom formula (none), so setting it to 4

52
3495
1.4349587988224717


In [20]:
# Fitting HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size = bx_cluster_size, 
                        metric = "euclidean",  # Euclidean metric best for staten island
                        prediction_data = True,
                        cluster_selection_epsilon = 0.05)

cluster_labels_train = cluster.fit_predict(X_train_scaled)

c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [21]:
# Assigning cluster labels to training data
st_train["cluster"] = cluster_labels_train

# Assigning cluster labels to test data with approximate predictions
cluster_labels_test, _ = hdbscan.approximate_predict(cluster, X_test_scaled)
st_test["cluster"] = cluster_labels_test

## Manhattan Clustering

In [22]:
# Subsetting to only Manhattan
mtn_df = listings[listings["boro"] == "Manhattan"]

# Train-Test Splits by index
mtn_train_idx, mtn_test_idx = train_test_split(mtn_df.index, train_size = 0.8, random_state = 2025)
mtn_train = mtn_df.loc[mtn_train_idx]
mtn_test  = mtn_df.loc[mtn_test_idx]

# Getting Lat/Lon as NumPy Arrays
X_train = mtn_train[['lat', 'lon']].values
X_test  = mtn_test[['lat', 'lon']].values

# Scaling Lat/Lon coordinates
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [23]:
# Getting unique neighborhoods
print(mtn_df["nbhd"].nunique()) #  unique neighborhoods in Manhattan
print(len(mtn_df)) # 107497 unique listings in Manhattan
print((len(mtn_df) / len(listings)) * 100) # 44.14% of listings are in Manhattan

mtn_cluster_size = 44 # Setting number of clusters to percentage of listings

333
107497
44.135555363954


In [24]:
# Fitting HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size = mtn_cluster_size, 
                        metric = "euclidean", # The Manhattan metric is best for the grid-like pattern that Manhattan has
                        prediction_data = True, 
                        cluster_selection_epsilon = 0.05) 

cluster_labels_train = cluster.fit_predict(X_train_scaled)

c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\adria\anaconda3\envs\analytics\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [25]:
# Assigning cluster labels to training data
mtn_train["cluster"] = cluster_labels_train

# Assigning cluster labels to test data with approximate predictions
cluster_labels_test, _ = hdbscan.approximate_predict(cluster, X_test_scaled)
mtn_test["cluster"] = cluster_labels_test

## Concatenating Data

In [26]:
# Making a function to label clusters in each borough
def add_prefix(df, boro_code):
    df["cluster"] = df["cluster"].astype(str)
    df.loc[df["cluster"] != "-1", "cluster"] = boro_code + "_" + df["cluster"]
    return df

In [27]:
# Adding prefixes to each borough and concatenating trained clusters and approximated clusters
qns_final = add_prefix(pd.concat([qns_train, qns_test]), "qns")
bk_final  = add_prefix(pd.concat([bk_train, bk_test]), "bk")
bx_final  = add_prefix(pd.concat([bx_train, bx_test]), "bx")
st_final  = add_prefix(pd.concat([st_train, st_test]), "st")
mtn_final = add_prefix(pd.concat([mtn_train, mtn_test]), "mtn")

# Making a final lat/lon df
final = pd.concat([qns_final, bk_final, bx_final, st_final, mtn_final])

In [28]:
# Viewing final df of clustered features
final

,accomm,avail_30,avail_365,bathrooms,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,...,aug,prop_apt,prop_hotel,prop_other,prop_vacation_home,prop_entire_home_apt,prop_hotel_room,prop_private_room,prop_shared_room,cluster
121864,2,29,89,1.0,1.0,2.0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,qns_14
189604,5,29,364,1.0,3.0,3.0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,qns_88
118835,2,17,138,1.0,1.0,1.0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,qns_57
104204,4,12,225,1.0,2.0,2.0,1,1,0,0,...,0,1,0,0,0,1,0,0,0,qns_88
214543,1,20,139,1.0,1.0,1.0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,qns_88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195516,6,29,364,1.0,2.0,3.0,6,6,0,0,...,0,1,0,0,0,1,0,0,0,mtn_275
84440,3,9,244,1.0,1.0,1.0,1,1,0,0,...,0,1,0,0,0,1,0,0,0,-1
54833,2,30,89,1.0,1.0,1.0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,-1
103334,2,30,89,1.0,4.0,4.0,25,24,1,0,...,0,1,0,0,0,0,0,1,0,-1


In [29]:
# Viewing amount of "noise" decided by HDBSCAN
final[final["cluster"] == "-1"].shape[0]

34656

In [30]:
# Writing to CSV
final.to_csv("clustered_listings.csv")